# 文字列処理

パソコンで扱うファイルは，テキストファイルとバイナリファイルに大別される．テキストファイル内に記載されている文字列情報の中から，目的の情報を抽出することが有益となることが多い．また，頻繁にその対応に迫られる．

生命科学においては，様々なフォーマットに様々な情報が記載されており，一つのファイルで全ての情報を得ることが容易ではない．多くの場合，複数のファイルから相互参照などを行うことで目的の情報を抽出することになる．

そのためにも，Pythonなどのスクリプト言語によって，テキストファイル処理を習得することは，生物情報をより広範に整理・収集する有効な手立てとなる．

## テキストファイル

テキストファイル (Text File) は，文字など文字コードによって表されるデータだけが含まれるファイルのことで，ファイルフォーマットの一種と見なすこともできる．互換性が高く幅広い環境でデータを利用できる利点がある一方，単純な文字だけしか扱えないという制限がある．

テキストファイル処理時には，ファイルの構造を理解しておくことが重要となる．例えば，塩基位置は，ファイル形式ごとに最初の塩基を0番目と扱うのか，1番目と扱うかの違いがある．従って，それぞれのファイル形式の定義を参照して，思い通りの情報を得られているかについては，その都度しっかりと確認する必要がある．

生命情報で頻出するファイル形式としては，以下ものが挙げられる．

| ファイル形式 | 格納情報 | 参照 |
|:---|:---|:---|
|FASTA| DNAやアミノ酸などの配列データ |https://ja.wikipedia.org/wiki/FASTA |
| GFF3 File Format | 遺伝子アノテーション情報 | http://gmod.org/wiki/GFF3　<br> https://github.com/The-Sequence-Ontology/Specifications/blob/master/gff3.md |
|SAM (Sequence Alignment/Map Format)|NGSリードのマッピング結果| https://samtools.github.io/hts-specs/SAMv1.pdf <br> https://cell-innovation.nig.ac.jp/surfers/SAM.html |
| FASTQ |DNAなどの塩基配列とそのクオリティスコア| https://ja.wikipedia.org/wiki/Fastq |

GFF3やSAMのように，複数列から成るテキストファイルを扱うことが多い．列の区切り文字としては，``タブ(\t)``，``スペース( )``，``comma(,)``が主に用いられている．

## バイナリファイル

テキストとはデータの内容すべてを人間が読んで理解できる (human-readable) もの，バイナリとはそうでないものを指す．
例えば，画像ファイル，音声ファイル，圧縮ファイルなどはバイナリファイルである．


| ファイル形式 | 説明 | 参照 |
|:---|:---|:---|
| gz | 「gzip」コマンドで圧縮した圧縮ファイルに付く拡張子 | https://ja.wikipedia.org/wiki/Gzip |
|BAM (Binary Alignment Map)|NGSリードのマッピング結果 | https://samtools.github.io/hts-specs/SAMv1.pdf <br> https://cell-innovation.nig.ac.jp/surfers/SAM.html |


# ファイルの読み書き

プログラムで処理した結果を保存しておきたい場合や，外部からPythonにデータを取り込む場合にファイルを使用する．

Pythonでファイルを操作するためには，組み込み型の**ファイル型**を使うことができます．特別な宣言は不要で，**open**関数を使用して，ファイルの読み書きを行う．open関数は，ファイルを開き、対応するファイルオブジェクトを返します．ファイルを開くことができなければ、OSErrorが返されます．

``open(filename, mode)``
- `filename`は，ファイルに付けられた名前の文字列
- `mode`は，ファイルのタイプやファイルをどのように操作したいかを知らせるための文字列

`open`でファイルを開いたら，`close`によってファイルを閉じる必要があります．しかしながら，`with文法`を使うことで，`close`処理が不要になります．`with文`では，ファイルが存在する場合はファイルを開いて，処理を実行し，最後に`close`処理も実行されます．
ファイルが存在しない場合には，実行しない仕組みになっています．

|mode|説明|
|:---|:---|
|r|ファイルを読み込み専用で開く (デフォルト)|
|w|ファイルを書き出し専用で開く|
|a|ファイルの最後に追記|
|x|ファイルを書き出し専用で開く<br>ファイルが存在しない場合のみ|
|+|読み込みと書き込みの両方を可能する <br> 「r+」「w+」|
|b|ファイルをバイナリモードで開く|


### ファイルを読み込む

ファイルの読み込みは，`open`を使用する．`as`に続く変数にファイルオブジェクトが代入されます．


```python
with open (filename, 'r') as f: # 'r'は省略化
```

### ファイルに書き込む

ファイルの書き込みは，読み込み同様`open`を使用する．ただし，`mode='w'`とする．

```python
with open (filename, 'w') as f:
```

### 改行コード

**改行コード**とは，改行を表す制御文字である．文字列の中で改行がある部分に対して，改行を指示する文字コードになります．OSで改行コードが異なる．

|改行コード|見え方|System|
|:---|:---|:---|
| CR | \r | mac|
| LF |\n|unix|
| CR+LF |\r\n|win|

## ファイル読み込み

酵母のgffファイル's288c_n20.gff'（最初の20行だけを抽出）を題材として，ファイル読み込みを行います．

In [ ]:
#ファイルの中身を確認する
%cat ./input/s288c_n20.gff

GFF3は，9列からなるテキストファイルで，
各列は，``タブ(\t)``で区切られている．

|列数|名前|情報|例（19行目）|
|:---|:---|:---|:---|
|1|seqid|染色体名やスキャッフォールド名|NC_001133.9|
|2|source|プロジェクト名やソフトウエア名など自由記載|RefSeq|
|3|type|属性型|CDS|
|4|start|開始位置|2480|
|5|end|終点位置|2707|
|6|score|スコア（.はスコアなし）|.|
|7|strand|ストランドの向き|+|
|8|phase|読み枠に関する情報（0,1,2）<br> CDS featureの場合には必須|0|
|9|attributes|付属情報（``セミコロン(;)``区切り）<br>パーセントエンコーディング（URL エンコード）|ID=cds1;Parent=rna1;Dbxref=SGD:S000028593,GeneID:1466426...|

19行目のattributesは以下のようになっている．
`ID=cds1;Parent=rna1;Dbxref=SGD:S000028593,GeneID:1466426,Genbank:NP_878038.1;Name=NP_878038.1;Note=hypothetical protein%3B identified by gene-trapping%2C microarray-based expression analysis%2C and genome-wide homology searching;gbkey=CDS;product=hypothetical protein;protein_id=NP_878038.1`

|タグ|情報|
|:---|:---|
|ID|cds1|
|Parent|rna1|
|Dbxref|SGD:S000028593,GeneID:1466426,Genbank:NP_878038.1|
|Name|NP_878038.1|
|Note|hypothetical protein%3B identified by gene-trapping%2C microarray-based expression analysis%2C and genome-wide homology searching|
|gbkey|CDS
|product|hypothetical protein|
|protein_id|NP_878038.1|

**Note**はURLエンコーディンク記法が採用されている．
- ; semicolon (%3B)
- = equals (%3D)
- & ampersand (%26)
- , comma (%2C)

|URLエンコード|URLデコード|
|:---|:---|
|hypothetical protein%3B identified by gene-trapping%2C microarray-based expression analysis%2C and genome-wide homology searching|hypothetical protein; identified by gene-trapping, microarray-based expression analysis, and genome-wide homology searching|

### 読み込み1

早速，ファイルを読み込んでみよう．

- 's288c_n20.gff'を読み込んで出力する．一行目だけ出力する．

In [ ]:
input_gff = "./input/s288c_n20.gff"
with open(input_gff) as f: #ファイルオープン
    line = f.readline()  # 一行読み込み
    print(line)        # 出力

1行目`##gff-version 3`が，読み込こまれました．`readline()`メソッドは，ファイルから1行読み込み，文字列を返します．テキストファイルをプログラムで処理する場合は，1行ごとに読み込んで処理することが多いことから，`readline()`メソッドを用いた例を紹介しています．`read()`メソッドは，ファイル全体を読み込むことが可能ですが，ファイルサイズが大きい場合には，注意が必要です．

### 読み込み2

1行目だけではなく，ファイル全体を読み込みましょう．
- s288c_n20.gffを読み込んで出力する．

In [ ]:
with open(input_gff) as f:
    for line in f:     # 一行ずつ読み込み
        print(line)    # 出力

`for`ループを使い，ファイルから1行ずつ読み込み，出力しました．出力結果には空行が追加されているはずです．`print(line, end='') `とすることで，改行コードを付与しない出力も可能ですが，その都度気にして作業することはあまりないと思います．

### 読み込み3 

- s288c_n20.gffを読み込んで出力する．改行コード削除を実行する．

In [ ]:
with open(input_gff) as f:
    for line in f:
        line = line.rstrip()  # 改行コードの削除
        print(line)

これで，元のファイルと同じ出力結果となりました．`str.rstrip()`メソッドは，引数に何も指定しない場合は，文字列の末尾部分の空白文字（スペース (space)、タブ (tab)、改行 (linefeed)、復帰 (return)、改頁 (formfeed)、垂直タブ (vertical tab) ）を除去します．あくまで好みの問題ですが，筆者は見えない末尾の空白文字は，最初に削除して処理することを心がけています．

### 読み込み4

目的とする情報のみを抽出することが可能です．
- s288c_n20.gffを読み込んで出力する．改行コード削除を実行する．``#``で始まる行だけを出力する．

In [ ]:
with open(input_gff) as f:
    for line in f:
        line = line.rstrip()
        
        if line.startswith('#'):  # ’#’で始まるかどうか
            print(line)

`#`で始まるヘッダー行だけを抽出することができました．

### 読み込み5

- s288c_n20.gffを読み込んで出力する．改行コード削除を実行する．``#``で始まる行以外を出力する．

In [ ]:
with open(input_gff) as f:
    for line in f:
        line = line.rstrip()
        
        if line.startswith('#'): # ’#’で始まるかどうか
            continue
        print(line)

### 読み込み6

- s288c_n20.gffを読み込んで出力する．改行コード削除を実行する．``#``で始まる行以外を出力する．9列目のデータのみを出力する．

In [ ]:
with open(input_gff) as f:
    for line in f:
        line = line.rstrip()

        if line.startswith('#'):
            continue
        s = line.split('\t')  # タブで区切る
        print(s[8])         # 9列目を出力

### 読み込み7

- s288c_n20.gffを読み込んで出力する．改行コード削除を実行する．``#``で始まる行以外を出力する．9列目のデータのみを出力する．`;`で区切る．

In [ ]:
with open(input_gff) as f:
    for line in f:
        line = line.rstrip()
        if line.startswith('#'):
            continue
        
        s = line.split('\t')
        items = s[8].split(';') # 9列目を”;”で区切る
        for item in items:    # リストを一つずつ
            print(item)       # 出力

### 読み込み8

- s288c_n20.gffを読み込んで出力する．改行コード削除を実行する．``#``で始まる行以外を出力する．9列目のデータのみを出力する．`;`で区切る．`product`を抽出する．

In [ ]:
with open(input_gff) as f:
    for line in f:
        line = line.rstrip()
 
        if line.startswith('#'):
            continue
        s = line.split('\t')
        items = s[8].split(';')
        for item in items:
            if item.startswith('product='):
                print(item)

### 読み込み9

- s288c_n20.gffを読み込んで出力する．改行コード削除を実行する．``#``で始まる行以外を出力する．9列目のデータを全て辞書型で取得する．

In [ ]:
with open(input_gff) as f:
    for line in f:
        line = line.rstrip()
 
        if line.startswith('#'):
            continue
        s = line.split('\t')
        items = s[8].split(';')
        tags = dict([(tmp.split('=')) for tmp in items]) # リスト内包表記で一括取得
        print(tags)

### 読み込み10

- s288c_n20.gffを読み込んで出力する．改行コード削除を実行する．``#``で始まる行以外を出力する．9列目の`Note`データのみを出力する．

In [ ]:
with open(input_gff) as f:
    for line in f:
        line = line.rstrip()
 
        if line.startswith('#'):
            continue
        s = line.split('\t')
        items = s[8].split(';')
        tags = dict([(tmp.split('=')) for tmp in items]) 

        if 'Note' in tags:      # Noteタグの有無
            print(tags['Note']) # Noteの内容を出力

### 読み込み11

- s288c_n20.gffを読み込んで出力する．改行コード削除を実行する．``#``で始まる行以外を出力する．9列目の`Note`データをURLデコードして，出力する．

In [ ]:
import urllib.parse   # urllib.parseモジュールをインポート
with open(input_gff) as f:
    for line in f:
        line = line.rstrip()
 
        if line.startswith('#'):
            continue
        s = line.split('\t')
        items = s[8].split(';')
        tags = dict([(tmp.split('=')) for tmp in items]) 

        if 'Note' in tags:     
            print(urllib.parse.unquote(tags['Note'])) # Noteの内容をデコードして出力

urllib.parseモジュールを使えば，URLエンコード，デコードが実現できます．

### 読み込み その他

2019年度の講習会時に頂いた質問です．
- ファイルの3行目から読み込みたい場合は，どのようにすれば良いでしょうか？

In [ ]:
with open(input_gff) as f:
    f.readline()      # 1行目の読み込みを実行
    f.readline()      # 2行目の読み込みを実行
    for line in f:    # 3行目からの読み込みを実行
        line = line.rstrip()
        print(line)

`readline()`を2回実行することで，最初の2行を読み込んだことになります．その後に，`for`ループを用いることで，3行目からの処理が可能となりました．

## アノテーションデータの処理例

テキストファイル処理の例として，遺伝子アノテーション情報を例にする．`EggNOG-mapper`は，遺伝子機能のアノテーションツールの一つであり，多岐に渡る情報が付与される．  

- eggNOG_OGs
- COG_cat
- Preferred_name
- GOs
- EC
- KEGG_ko
- KEGG_Pathway
- KEGG_Module
- KEGG_Reaction
- KEGG_rclass
- BRITE
- KEGG_TC
- CAZy
- BiGG_Reaction
- PFAMs
などの情報が付与された結果ファイルを得ることができる．

大腸菌のアミノ酸配列を`EggNOG-mapper`にかけて得られた結果ファイル`ecoli_eggnog.emapper.annotations`を使用して進める．

In [ ]:
!head input/ecoli_eggnog.emapper.annotations

### CAZy

CAZyは，carbohydorateの合成，修飾，分解それぞれの酵素情報のデータベースである．  
http://www.cazy.org

結果ファイルから，どの酵素がいくつ検出されたのかを集計してみる．  
CAZyの情報は，19列目に記載されている．

ヒットしなかった場合は，`'-'`，複数のヒットがある場合は，`','`で区切られている．

In [ ]:
input_ant = 'input/ecoli_eggnog.emapper.annotations'

cazy = {}

with open(input_ant) as f:
    for line in f:
        line = line.rstrip()
        
        if line.startswith('#'):
            continue
            
        s = line.split('\t')
        
        caz = s[18]
        
        if caz == '-':
            continue
        
        for item in caz.split(','):
            if caz in cazy:
                cazy[item] += 1
            else:
                cazy[item] = 1

In [ ]:
cazy

`GT2 (GlycosylTransferase Family 2)`の酵素が9つあることが分かった．  
結果を見やすくするため，数が多い順で並び替えてみる．

In [ ]:
sorted(cazy.items(), key=lambda x:x[1], reverse=True)

### COG

COGは，遺伝子の機能分類に関するデータベースである．
https://www.ncbi.nlm.nih.gov/research/cog/

結果ファイルから，どの酵素がいくつ検出されたのかを集計してみる．  
COGの情報は，7列目に記載されている．

ヒットしなかった場合は，`'-'`，ヒットした場合は，文字（列）で表現されている．  
各文字と機能の関係については下記の表の通りである．

|Functional category ID <br> (one letter)| Functional category description|
|:--:|:---|
|J|	Translation, ribosomal structure and biogenesis|
|A|	RNA processing and modification|
|K|	Transcription |
|L|	Replication, recombination and repair |
|B|	Chromatin structure and dynamics|
|D|	Cell cycle control, cell division, chromosome partitioning|
|Y|	Nuclear structure|
|V|	Defense mechanisms|
|T|	Signal transduction mechanisms|
|M|	Cell wall/membrane/envelope biogenesis|
|N|	Cell motility|
|Z|	Cytoskeleton|
|W|	Extracellular structures| |
|U|	Intracellular trafficking, secretion, and vesicular transport|
|O|	Posttranslational modification, protein turnover, chaperones|
|X|	Mobilome: prophages, transposons|
|C|	Energy production and conversion|
|G|	Carbohydrate transport and metabolism|
|E|	Amino acid transport and metabolism|
|F|	Nucleotide transport and metabolism|
|H|	Coenzyme transport and metabolism|
|I|	Lipid transport and metabolism|
|P|	Inorganic ion transport and metabolism|
|Q|	Secondary metabolites biosynthesis, transport and catabolism|
|R|	General function prediction only|
|S|	Function unknown|

In [ ]:
input_ant = 'input/ecoli_eggnog.emapper.annotations'

COG = {'J':0, 'A':0, 'K':0, 'L':0, 'B':0, 
    'D':0, 'Y':0, 'V':0, 'T':0, 'M':0,
    'N':0, 'Z':0, 'W':0, 'U':0, 'O':0,
    'X':0, 'C':0, 'G':0, 'E':0, 'F':0,
    'H':0, 'I':0, 'P':0, 'Q':0, 'R':0,
    'S':0}

with open(input_ant) as f:
    for line in f:
        line = line.rstrip()
        
        if line.startswith('#'):
            continue
            
        s = line.split('\t')
        
        cog = s[6]
        
        if cog == '-':
            continue
        
        for item in cog:
            if item in COG:
                COG[item] += 1

In [ ]:
COG

同様に，数の多い順に並べ替えてみる．

In [ ]:
sorted(COG.items(), key=lambda x:x[1], reverse=True)

## 書き込み

### 書き込み1

`open(file, mode='w')`とすれば，ファイルへの書き込みができます．
- "Hello world!\n"をtest1.txtに出力する．

In [ ]:
output = "./output/test1.txt"   # ファイル名
out = "Hello world!\n"          # 出力
with open(output, 'w') as f:  # ファイルオープン
    f.write(out)              # ファイルへ書き込み

`test1.txt`ファイルが`output`ディレクトリ下に作成されました．中身を確認すると，`Hello world!`となっています．
`output`ディレクトリがない場合は，`FileNotFoundError: No such file or directory: './output/test1.txt'`となります．

`mode`オプションを変更することで，上書きするのか，追記するのか，新規作成するのかが設定できます．

### 書き込み2

- "Hello world!\n"をtest1.txtに出力する．ファイルが存在する場合は，上書きしない．

In [ ]:
output = "./output/test1.txt"   # ファイル名
out = "Hello world!\n"          # 出力
with open(output, 'x') as f:  # ファイルオープン
    f.write(out)              # ファイルへ書き込み

`FileExistsError: File exists: './output/test1.txt'`となりました．`mode=x`を用いることで，既存のファイルへの上書きを防ぐことができます．

## SAM

SAMファイルは，`@`で始まるヘッダー行と各リードのアライメント情報を格納している．アライメントに関する情報は，11列には固有の情報が記述されている．12列目以降は，付属情報となる．

|列数|Field|情報|
|:---|:---|:---|
|1|QNAME|リード名|
|2|FLAG|アライメント情報|
|3|RNAME|リファレンス名|
|4|POS|マッピング位置|
|5|MAPQ|マッピングスコア|
|6|CIGAR|マッピングの状況| 
|7|RNEXT|ペアエンドリードのマッピングされたリファレンス名<br> '='の場合は，同じリファレンス名|
|8|PNEXT|ペアエンドリードのマッピング位置|
|9|TLEN|リード間の距離|
|10|SEQ|リードの塩基配列|
|11|QUAL|塩基配列のクオリティデータ|

In [ ]:
#ファイルの中身を確認する
!head -n 25 ./input/SRR453566.sam

### ビット演算子

2列目のFLAG情報は，リードのマッピング状況を知ることができる．各情報は下記のように定義されている．

|10進数|Bit|説明（SAM定義）|Decoding SAM flags|
|:--|:--|:--|:--|
|1 | 0x1 |template having multiple segments in sequencing|read paired|
|2 | 0x2 |each segment properly aligned according to the aligner | read mapped in proper pair |
|4 | 0x4 |segment unmapped| read unmapped |read unmapped |
|8 | 0x8 |next segment in the template unmapped |  mate unmapped|
|16 | 0x10 |SEQ being reverse complemented| read reverse strand |
|32 | 0x20 |SEQ of the next segment in the template <br> being reverse complemented | mate reverse strand |
|64 | 0x40 |the first segment in the template| first in pair |
|128 | 0x80 |the last segment in the template | second in pair |
|256 | 0x100 |secondary alignment| not primary alignment | 
|512 | 0x200 |not passing filters, such as platform/vendor quality controls | read fails platform/vendor quality checks|
|1024 | 0x400 |PCR or optical duplicate|  read is PCR or optical duplicate |
|2028| 0x800 |supplementary alignment | supplementary alignment |

FLAG情報に関しては，`Decoding SAM flags`サイトが分かりやすい．
https://broadinstitute.github.io/picard/explain-flags.html


FLAG`83`を持つリードは，`read paired (0x1)`，`read mapped in proper pair (0x2)`，`read reverse strand (0x10)`，`first in pair (0x40)`，でリードがマッピングされていることを示している．幾つかのFLAGについて，bit情報をまとめた．



|10進数|Bit|FLAG=83|FLAG=163|FLAG=77|FLAG=137|
|:--:|:--:|:--:|:--:|:--:|:--:|
|1 | 0x1 | &#9675;|&#9675;|&#9675;|&#9675;|
|2 | 0x2 | &#9675;|&#9675;|||
|4 | 0x4 |||&#9675;||
|8 | 0x8 |||&#9675;|&#9675;|
|16 | 0x10 |&#9675;	||||
|32 | 0x20 ||&#9675;|||
|64 | 0x40 |&#9675;	||&#9675;||
|128 | 0x80 ||&#9675;||&#9675;|
|256 | 0x100 |||||
|512 | 0x200 |||||
|1024 | 0x400 |||||
|2028| 0x800 |||||



ビット演算子を用いて，FLAG情報を扱うことができる．
例えば，`if FLAG & 0x4:`
とすることで，unmappedかどうかの判定ができる．

| ビット演算子 | 説明 |
|:-----:|:-----:|
|x &#124; y|xとyの論理和&#40;OR&#41;を取る|
|x &amp; y|xとyの論理積&#40;AND&#41;を取る|
|x^y|xとyの排他的論理和&#40;XOR&#41;を取る|


### SAM1

- FLAGの状況を確認する．

In [ ]:
input_sam = "./input/SRR453566.sam"
dic = {}
with open(input_sam) as f:
    for line in f:
        line = line.rstrip()
        if line.startswith('@'): # ヘッダー行をスキップ
            continue
        s = line.split('\t')       # タブで区切る
        FLG = s[1]                 # 2列目を格納
        if FLG in dic:           # 辞書に格納
            dic[FLG] += 1
        else :
            dic[FLG] = 1

# 出力 FLAG, 総数
for k, v in dic.items():
    print(k+'\t'+str(v))

FLAG情報の集計結果が得られた．例えば，FLAG`83`のリードが197本あったことが分かる．

### SAM2

- 特定のFLAGを有するリード情報を抽出する．
マッピングされなかったリードを抽出する．``read unmapped``は，``0x4``，``4``で表現される．

In [ ]:
readList = []
with open(input_sam) as f:
    for line in f:
        line = line.rstrip()
        if line.startswith('@'):
            continue
        s = line.split('\t')
        FLG = int(s[1])
        if FLG & 0x4:
            readList.append(s[0])

# 出力
for item in readList:
    print(item)

ビット演算子を用いて，`read unmapped (0x4)`ビットをもつFLAG`69`，`77`，`133`，`141`のリードが抽出された．`samtools`を使うことで，同様の処理が可能となる．

# 正規表現

**正規表現**（Regular Expression）とは、「文字列の集合を一つの文字列で表現する方法の一つである」<sup>[1]</sup>．通常の文字列とメタ文字と呼ばれる特殊な文字を組み合わせてパターンを作り，パターンに指定された法則で並ぶ文字列検索を実現できる正規表現は，ファイルや他のデータの中から，複雑な文字列のパターンを検索するのに役立ちます．

1. https://ja.wikipedia.org/wiki/正規表現

制御配列の一つTATAボックスを例に，正規表現を試してみる．
TATAボックスは，RNAポリメラーゼIIによる転写開始位置の上流25塩基対の位置，あるいはさらに上流に存在する共通した塩基配列で，
5'-TATA[A/T]AA[G/A]-3'
と定義されている<sup>[2]</sup>．

2. Smale ST & Kadonaga JT. The RNA polymerase II core promoter. Annu Rev Biochem. 2003;72:449-79.

TATAボックスを有するリードをSAMファイルから検索する．
文字列一致で探索する場合は，4通りの文字列を探索する必要がある．
相補鎖配列も含めると，計8通りの文字列の探索が必要となる．

- 5'-TATA**[A]**AA**[G]**-3'
- 5'-TATA**[A]**AA**[A]**-3'
- 5'-TATA**[T]**AA**[A]**-3'
- 5'-TATA**[T]**AA**[G]**-3'

正規表現は，標準ライブラリの`reモジュール`を使います．


| メソッド | 説明 |
|:-----|:-----|
|compile|正規表現patternのオブジェクトを生成|
|match|文字列の先頭での正規表現patternの有無を探索|
|search|文字列内での正規表現patternの有無を探索|
|findall|文字列内での全ての正規表現patternを探索|


正規表現で使用する特殊文字を一部紹介します．

| パターン | 説明 |
|:-----|:-----|
|\d|1個の数字|
|\D|1個の数字以外の文字|
|\w|1個の英字|
|\W|1個の英字以外の文字|
|\s|1個の空白文字|
|\S|1個の空白文字以外の文字|
|.|\n以外の任意の文字|
|^|文字列の先頭|
|$|文字列の末尾|
|*|直前の正規表現を0回以上，できるだけ多く繰り返したもの．<br>ab*は'a'，'ab'，または'a'に任意個数の'b'を続けたもの．|
|+|直前の正規表現を0回以上，できるだけ多く繰り返したもの．<br>ab+は'ab'，または'a'に任意個数の'b'を続けたもの．|
|?|直前の正規表現を0回もしくは1回．<br>ab?は'a'あるいは'ab'．|
|[ ]|文字の集合．<br>[abc]は'a'または'b'または'c'．|

### 正規表現1

- TATAボックスを有する配列を探索する．reモジュールを使わずに，TATAボックスを探索する．4種類の配列それぞれの有無について調べる．

In [ ]:
input_sam = "./input/SRR453566.sam"
TATA1 = 'TATAAAAG'
TATA2 = 'TATAAAAA'
TATA3 = 'TATATAAG'
TATA4 = 'TATATAAA'

with open(input_sam) as f:
    for line in f:
        line = line.rstrip()
        if line.startswith('@'): # ヘッダー行をスキップ
            continue
        s = line.split('\t')       # タブで区切る
        sequence = s[9]            # 10列目を格納

        if TATA1 in sequence:
            print("TATA1:"+sequence)
            
        if TATA2 in sequence:
            print("TATA2:"+sequence)
            
        if TATA3 in sequence:
            print("TATA3:"+sequence)
            
        if TATA4 in sequence:
            print("TATA4:"+sequence)

4通りのTATAボックスを用意して，それぞれについて検索を行いました．相補鎖についても探索が必要な場合は，4通りの配列を加えてそれぞれに対して同様の処理を行えばよい．

### 正規表現2

- TATAボックスを有する配列を探索する．reモジュールによる正規表現パターンを探索する．

In [ ]:
import re

TATA = re.compile('TATA[AT]AA[GA]') # patternをコンパイル

with open(input_sam) as f:
    for line in f:
        line = line.rstrip()
        if line.startswith('@'): # ヘッダー行をスキップ
            continue
        s = line.split('\t')       # タブで区切る
        sequence = s[9]            # 10列目を格納

        m = TATA.search(sequence)  # patternを検索
        
        if m:
            print(sequence)

TATAボックス，``5'-TATA[A/T]AA[G/A]-3'``，を
`TATA=re.compile('TATA[AT]AA[GA]')`
としてコンパイルしている．これにより，効率的な処理が実現できる．
コンパイルしなくても，`re.search('TATA[AT]AA[GA]', 文字列)`とすることで，同様の処理が実現できる．

次に，``pattern.match(文字列)``として，正規表現パターンを検索している．この場合，計4種類のTATAボックス配列を検索している．マッチオブジェクトから，一致箇所の情報を抽出できる．

### 正規表現3

- TATAボックスを有する配列を探索する．reモジュールによる正規表現パターンを探索する．一致箇所を出力する．

In [ ]:
import re

TATA = re.compile('TATA[AT]AA[GA]') # patternをコンパイル

with open(input_sam) as f:
    for line in f:
        line=line.rstrip()
        if line.startswith('@'): # ヘッダー行をスキップ
            continue
        s = line.split('\t')       # タブで区切る
        sequence = s[9]            # 10列目を格納

        m = TATA.search(sequence)  # patternを検索
        
        if m:
            print(m.start(), m.end(), m.string[m.start():m.end()])

マッチオブジェクトを用いることで，一致した箇所の情報や一致した文字列の抽出が可能となる．`m.start()`，`m.end()`はマッチした部分文字列の先頭と末尾のインデックスを返します．

### 正規表現4

- TATAボックスを有する配列を探索する．reモジュールによる正規表現パターンを探索する．一致箇所を出力する．相補鎖についても探索する．

In [ ]:
import re

TATA = re.compile('TATA[AT]AA[GA]') # patternをコンパイル
TATA_rev = re.compile('[CT]TT[AT]TATA') # 相補鎖patternをコンパイル

with open(input_sam) as f:
    for line in f:
        line = line.rstrip()
        if line.startswith('@'): # ヘッダー行をスキップ
            continue
        s = line.split('\t')       # タブで区切る
        sequence = s[9]            # 10列目を格納

        m = TATA.search(sequence)  # patternを検索
        
        if m:
            print(m.start(), m.end(), m.string[m.start():m.end()])
            
        m = TATA_rev.search(sequence)  # 相補鎖patternを検索
        
        if m:
            print(m.start(), m.end(), m.string[m.start():m.end()])